In [5]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata
import os
os.environ["OPENAI_API_KEY"] = ''

In [6]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [7]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-05-11 09:16:07--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  3.72MB/s    in 0.5s    

2024-05-11 09:16:08 (3.72 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-05-11 09:16:08--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connec

In [8]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [9]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [10]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [11]:
from llama_index.agent.openai import OpenAIAgent

In [12]:
agent = OpenAIAgent.from_tools(query_engine_tools, verbose=True)

In [13]:
agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  What was Lyft's revenue growth in 2021?


Added user message to memory: What was Lyft's revenue growth in 2021?
=== Calling Function ===
Calling function: lyft_10k with args: {"input":"Lyft's revenue growth in 2021"}
Got output: Lyft's revenue increased by 36% in 2021 compared to the prior year.

Assistant: Lyft's revenue grew by 36% in 2021 compared to the previous year.



Human:  What was Uber's revenue growth in 2021 and can you please unpack the drivers of growth?


Added user message to memory: What was Uber's revenue growth in 2021 and can you please unpack the drivers of growth?
=== Calling Function ===
Calling function: uber_10k with args: {"input": "Uber's revenue growth in 2021"}
Got output: Uber's revenue grew by 57% in 2021 compared to the previous year.

=== Calling Function ===
Calling function: uber_10k with args: {"input": "Drivers of Uber's revenue growth in 2021"}
Got output: The drivers of Uber's revenue growth in 2021 were primarily attributed to the increase in Gross Bookings, particularly driven by the growth in Delivery Gross Bookings and Mobility Gross Bookings. The growth in Delivery Gross Bookings was fueled by an increase in food delivery orders and higher basket sizes due to stay-at-home orders related to COVID-19, along with continued expansion in both U.S. and international markets. Additionally, the increase in Mobility Gross Bookings was a result of rising Trip volumes as the business recovered from the impacts of COVID

Human:  exit
